<a href="https://colab.research.google.com/github/luisantoniococa/Spark_Big_Data_Amazon_Reviews_Analysis/blob/master/Amazon_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [39]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-04-15 16:15:24--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2020-04-15 16:15:25 (8.52 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [41]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
jewelry_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Jewelry_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
jewelry_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...|2015-08-31 00:00:00|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...| 

In [42]:
# print the data schemata to identify the differents columns
jewelry_data_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [43]:
# Get the amount fo rows in the dataframe
jewelry_data_df.count()

1767753

In [44]:
# Drop the na values and drop duplicates values 
dropna_df = jewelry_data_df.dropna()
dropduplicates_df = dropna_df.dropDuplicates()

dropduplicates_df.count()

1767394

In [45]:
from pyspark.sql.functions import to_date
# review the DataFrame and transform the date column to date
review_id_df = dropduplicates_df.select(["review_id", "customer_id", "product_id", 
                                       "product_parent", 
                                       to_date("review_date", 'yyyy-MM-dd').alias("review_date")]) # using .alias works as in sql
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1001WBBDL0FZV|   23386199|B00EQDCO24|     864125801| 2014-07-31|
|R1004GO1PYBZ4B|   29873923|B00MUN8KHA|     635123005| 2015-06-02|
|R100MMJ11EQN2Y|   26499325|B007SYKBXM|     796678590| 2013-02-05|
|R100XMJO9EM9BA|    3275800|B00CDJX87S|     555898816| 2014-08-03|
|R1014FLXHNOPI0|   34768821|B002MQ9EBG|     864672827| 2015-05-25|
|R101Y79X8R3B0V|   11130117|B00MBCUCB2|     851328672| 2015-05-15|
|R102CKN6J3NJOA|   40395829|B00L365T7O|     543506306| 2015-06-04|
|R102G1MA1CDH0W|   53012317|B001L7PESE|     664095849| 2008-12-20|
|R102P1F7APXDMR|   24967469|B00CDGTHVW|     536118532| 2014-08-23|
|R102UX37P1SSZ9|   48462918|B00CGSXJH0|     255117861| 2015-08-29|
|R102WVZLK77W9O|   33180135|B003ZDOPRI|     697802402| 2013-10-12|
|R1032ANR3J8KTX|    2869948|B008BY9N0A|     642816768| 2014-08

In [0]:
# finding duplicates values by ID and dropping them

products_df = jewelry_data_df.select(['product_id', 'product_title'])
products_df = products_df.drop_duplicates(subset = ['product_id'])
# changed drop_duplicates() for dropDuplicates() important to add the subset 


In [52]:
new_df = products_df.groupBy('product_id').count().orderBy('count', ascending=False)
new_df.show()

+----------+-----+
|product_id|count|
+----------+-----+
|B00SCVJBAY|    1|
|B00QRJMR8Q|    1|
|B008BK2SCE|    1|
|B0119H2FEQ|    1|
|B0122U1X8S|    1|
|B00Y9FTS46|    1|
|B00O8XR67U|    1|
|B002AQ3K36|    1|
|B00BQQUW6E|    1|
|B007X2NVHC|    1|
|B00LKP6IQ4|    1|
|B00CWF9CQY|    1|
|B00HNGOC1W|    1|
|B00CZ7HO0U|    1|
|B00DS2PWZK|    1|
|B00N0SHG6U|    1|
|B001QVPNVI|    1|
|B00SPSV7HY|    1|
|B00TAH56N0|    1|
|B004QI8GMG|    1|
+----------+-----+
only showing top 20 rows



In [53]:
# check for duplicates since sql table has a unique condition for product_id Column
products_df.filter(products_df['product_id'] == 'B004NMC1M6').show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B004NMC1M6|Sterling Silver P...|
+----------+--------------------+



In [54]:
# reviews df for sql table 
reviews_df = jewelry_data_df.select(['review_id','review_headline','review_body'])
reviews_df.show(10)

+--------------+--------------------+--------------------+
|     review_id|     review_headline|         review_body|
+--------------+--------------------+--------------------+
|R135Q3VZ4DQN5N|           Beauties!|so beautiful even...|
|R2N0QQ6R4T7YRY|      Great product.|Great product.. I...|
|R3N5JE5Y4T6W5M|Exactly as pictur...|Exactly as pictur...|
|R2I150CX5IVY9Q|          Five Stars|Love it. Fits gre...|
|R1RM9ICOOA9MQ3|... a Mother's Da...|Got this as a Mot...|
|R2J2KMDL10UMSH|          Five Stars|I Love It Make Me...|
|R3R9ZUFA4TB4FQ|          Five Stars|True to size, unique|
|R3UQ8VAQN7R6WL|          Five Stars|This necklace is ...|
|R1FXZ69C01JNQM|          Five Stars|        just perfect|
| RY36LB4OW0FFS|          Five Stars|           Love it !|
+--------------+--------------------+--------------------+
only showing top 10 rows



In [61]:
customer_df = jewelry_data_df.groupby('customer_id').agg({'customer_id': 'count'}).withColumnRenamed('count(customer_id)','customer_count')
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   45657423|             1|
|   47663761|             1|
|   11931196|             1|
|   44620216|             1|
|    1167081|             1|
|   17682867|             1|
|   10782454|             1|
|   51881951|             1|
|    1537684|             1|
|   17133428|             5|
|     661760|             1|
|    9867391|             9|
|   31268594|             1|
|   48581590|             1|
|   32466409|             3|
|   38490515|             1|
|   41318181|             4|
|   18800155|             6|
|   46306239|             1|
|   28409312|             1|
+-----------+--------------+
only showing top 20 rows



In [62]:
customer_df.count()

1069439

In [57]:
vine_df = jewelry_data_df.select(['review_id','star_rating','helpful_votes','total_votes','vine'])
vine_df.show(10)


+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R135Q3VZ4DQN5N|          5|            0|          0|   N|
|R2N0QQ6R4T7YRY|          5|            0|          0|   N|
|R3N5JE5Y4T6W5M|          5|            0|          0|   N|
|R2I150CX5IVY9Q|          5|            0|          0|   N|
|R1RM9ICOOA9MQ3|          5|            0|          0|   N|
|R2J2KMDL10UMSH|          5|            0|          0|   N|
|R3R9ZUFA4TB4FQ|          5|            0|          0|   N|
|R3UQ8VAQN7R6WL|          5|            0|          0|   N|
|R1FXZ69C01JNQM|          5|            0|          0|   N|
| RY36LB4OW0FFS|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



## Pushing created dfs into AWS RDS postgres connected DB

In [0]:
mode = 'append'
jdbc_url="jdbc:postgresql://<endpoint>:5432/Amazon_reviews_db"
config = {"user":"<user>", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [0]:
# write the review_id_df to the table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)


In [0]:
# write the products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [0]:
# Write customers_df to table in RDS
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [0]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vines', mode=mode, properties=config)